# How to execute

To execute the application, you need to have a valid flow token. You can obtain this token using one of the following methods:

1. Generate a Flow Token Programmatically:
    Run the `flow_token.py script`. If the script executes successfully, it will create a `.env` file containing your **FLOW_TENANT** and **FLOW_TOKEN**.



2. Retrieve Your Flow Token from Browser Cookies:
    Extract your flow token directly from your browser cookies. **Ensure you are logged in with your CI&T account**. For guidance, refer to this [YouTube tutorial](https://youtube.com/clip/Ugkxsv6UO3IpQ2ZLfbb61z4QN90XohMfzh22?si=hYZao7TG1E1SwvSK).

In [1]:

import requests
import json
import threading
import os
import queue

from datetime import datetime
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

In [ ]:
flowToken = ""

In [2]:
#  model_engine = 'gemini-pro'
def chatCompletion_v1(prompt, output_queue):

    flow_llm_url = "https://flow.ciandt.com/ai-orchestration-api/v1/openai/chat/completions"
    flow_tenant = os.getenv("FLOW_TENANT") or "lithiadw"
    flow_token = os.getenv("FLOW_TOKEN") or flowToken

    if not flow_token:
        print("Flow token is missing. Please ensure FLOW_TOKEN environment variable is set.")
        return None

    payload = json.dumps({
        "stream": False,
        "max_tokens": 4096,
        "temperature": 0.7,
        "allowedModels": [
            "gpt-4o-mini"
        ],
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": prompt
                    }
                ]
            }
        ],
        
    })

    headers = {
        'Content-Type': 'application/json',
        'Accept': '*/*',
        'FlowTenant': flow_tenant,
        'FlowAgent': "Kanjih-Tools",
        'Authorization': f"Bearer {flow_token}",
    }

    print(f"""Requesting completion for: {prompt}""")

    r =  requests.request("POST", flow_llm_url, headers=headers, data=payload)
    print(f"Response: {r.status_code}")
    
    if(r.status_code != 200):
        print(f"{r.text} please recreate your flow token")
        return None
    
    # Create a queue to collect results
    result = {
        "prompt": prompt,
        "response": r.json()['choices'][0]['message']['content']
    }
    output_queue.put(result)
    




In [ ]:
promptsMessages = [
    "Define User Stories: What are the key user stories for this sprint?",
    "Identify Stakeholders: Who are the stakeholders involved in this project?",
    "Set Sprint Goals: What are the primary goals we want to achieve in this sprint?",
    "Create a Product Backlog: What features or tasks should be prioritized in our product backlog?",
    "Yesterday’s Progress: What did you accomplish yesterday?",
    "Today’s Focus: What will you be working on today?",
    "Identify Blockers: Are there any obstacles preventing you from completing your tasks?",
    "Task Estimation: How long do you estimate each task will take?",
    "Assign Tasks: Who will be responsible for each task in this sprint?",
    "Risk Assessment: What potential risks should we be aware of for this sprint?",
    "Feedback Loop: What feedback do you have for team members on their current tasks?",
    "Cross-Functional Collaboration: How can we better collaborate with other teams?",
    "Share Resources: What resources or tools do we need to improve our workflow?",
    "Update Task Status: What is the current status of your tasks? (To Do, In Progress, Done)",
    "Review Burndown Chart: How are we tracking against our sprint goals?",
    "Celebrate Wins: What successes have we achieved this sprint?",
    "What Went Well: What aspects of the sprint went well?",
    "Areas for Improvement: What could we improve for the next sprint?",
    "Action Items: What specific actions can we take to enhance our processes?",
    "Team Morale: How is everyone feeling about the current workload?",
    "Skill Development: What skills would you like to develop to contribute more effectively to the project?",
    "Conflict Resolution: Are there any conflicts within the team that need to be addressed?",
    "Process Review: Are there any processes we should consider changing?",
    "Innovation Ideas: What new ideas can we implement to enhance our product?",
    "Customer Feedback: How can we incorporate customer feedback into our next sprint?",
    "Long-Term Vision: What is our long-term vision for this project?",
    "Feature Prioritization: Which features should we focus on in the next few sprints?",
    "Budget Considerations: Are there any budget constraints we need to consider moving forward?",
    "Team Building: What team-building activities would you like to engage in?",
    "Recognition: Who would you like to recognize for their contributions this sprint?"
]



# Number of threads to create
num_threads = len(promptsMessages)
# Create a queue to collect results
output_queue = queue.Queue()

threads = []
for i in range(num_threads):
    # Create a thread, passing i+1 as the argument for print_numbers
    thread = threading.Thread(target=chatCompletion_v1, args=(promptsMessages[i], output_queue))
    threads.append(thread)
    thread.start()

# Wait for all threads to complete
for thread in threads:
    thread.join()

results = []
# Get all results from the queue
while not output_queue.empty():
    result = output_queue.get()
    results.append(result)

print(f"Results: {results}")

In [ ]:
results[0]
